In [1]:
import pandas as pd
import numpy as np
from pandas import Series, DataFrame
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from matplotlib import pyplot as plt
import pickle
import import_ipynb
import sys
sys.path.append('./../')
import Portfolio_value as pv
import statsmodels.api as sm

In [2]:
df =  pd.read_csv('./../data/data.csv')

In [3]:
p_val = np.load('./../result/p_val/p_val_dyna.npy', allow_pickle=True)
w_val = np.load('./../result/w_val/w_val_dyna.npy', allow_pickle=True)

p_val = p_val.reshape(1)
p_val = p_val[0]

w_val = w_val.reshape(1)
w_val = w_val[0]

In [4]:

# for upper_year in [15,20,25]:
#     for k in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
#         for withdraw_rate in [0.04,0.06,0.08,0.10]:
#             for year in range(1980,2003-upper_year):
             

In [5]:
normal_year = np.load('./../result/year/normal_year.npy', allow_pickle=True).reshape(1)[0]
bear_year = np.load('./../result/year/bear_year.npy', allow_pickle=True).reshape(1)[0]
full_year = np.load('./../result/year/full_year.npy', allow_pickle=True).reshape(1)[0]

In [6]:
upper_years =  [20, 25, 30]

In [7]:
normal_perform = {}
bear_perform = {}
full_perform = {}
for upper_year in upper_years:
    normal_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    bear_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}
    
    full_perform[upper_year] = {
    'dep':[],
    'max':[],
    'under':[],
    'inc':[],
    'beq':[],
    'total':[]
}

In [8]:
withdraw_rates = [0.04, 0.06, 0.08]

In [9]:
init_wealth = 350000
k = 0.5 # 고정

x = {
    20:[],
    25:[],
    30:[]
}
depletion = {
    20:[],
    25:[],
    30:[]
    
}
wealth = {
    20:[],
    25:[],
    30:[]
    
}




for withdraw_rate in [0.04, 0.06, 0.08]:
    for upper_year in upper_years:
        print('upper_year:', upper_year)
        for years in full_year[upper_year] :
            for year in years:
                print('year:', year)
                if year in normal_year[upper_year][0]:
                    x[upper_year].append(0)
                elif year in bear_year[upper_year][0]:
                    x[upper_year].append(1)

                res = pv.get_metric(year, k, upper_year, withdraw_rate, df, init_wealth, p_val, w_val)
                if res[2] < upper_year*12 : #고갈 시점이 upper_year보다 작으면 (이미 고갈이 되면)
                    dep_or_not = 1
                elif res[2] >= upper_year*12: # 고갈이 안되면
                    dep_or_not = 0
                    
                depletion[upper_year].append(dep_or_not)
                wealth[upper_year].append((res[1] + res[0])*350000/100/1000)
            



upper_year: 20
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
upper_year: 25
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
upper_year: 30
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
upper_year: 20
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
year: 1987
year: 1988
year: 1989
year: 1990
year: 1991
year: 1992
year: 1993
year: 1994
year: 1995
year: 1996
year: 1997
year: 1998
year: 1999
year: 2000
year: 2001
year: 2002
upper_year: 25
year: 1980
year: 1981
year: 1982
year: 1983
year: 1984
year: 1985
year: 1986
y

In [10]:
intercept_linear = {}
beta_linear = {}

intercept_logit = {}
beta_logit = {}
t_intercept_logit = {}
t_beta_logit = {}
r2 = {}

intercept_total = {}
beta_total = {}
t_intercept_total = {}
t_beta_total = {}
r2_total = {}        
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)
    intercept_linear[upper_year] = np.round(res.params[1],3)
    beta_linear[upper_year] = np.round(res.params[0],3)
    
#     t_intercept[upper_year] = np.round(res.tvalues[1],3)
#     t_beta[upper_year] = np.round(res.tvalues[0],3)
    

for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.Logit(depletion[upper_year], x_)#sm.OLS(depletion[upper_year], x_)
    
    try:
    
        res = mod.fit()
        print(res.summary())
        intercept_logit[upper_year] = np.round(res.params[1],3)
        beta_logit[upper_year] = np.round(res.params[0],3)
    
        t_intercept_logit[upper_year] = np.round(res.tvalues[1],3)
        t_beta_logit[upper_year] = np.round(res.tvalues[0],3)
        r2[upper_year] = np.round(res.prsquared, 3)
        print(res.prsquared)
    except:
        intercept_logit[upper_year] = '-'
        beta_logit[upper_year] = '-'
    
        t_intercept_logit[upper_year] = '-'
        t_beta_logit[upper_year] = '-'
        r2[upper_year] = '-'

    
for upper_year in upper_years:
    print('upper_year:', upper_year)
    x_ = sm.add_constant(x[upper_year], prepend=False)
    mod = sm.OLS(wealth[upper_year], x_)
    res = mod.fit()
    print(res.summary())
#     print(res.prsquared)

    intercept_total[upper_year] = np.round(res.params[1],3)
    beta_total[upper_year] = np.round(res.params[0],3)
    
    t_intercept_total[upper_year] = np.round(res.tvalues[1],3)
    t_beta_total[upper_year] = np.round(res.tvalues[0],3)
    
    r2_total[upper_year] = np.round(res.rsquared_adj, 3)

upper_year: 20
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.174
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     14.10
Date:                Mon, 10 Jul 2023   Prob (F-statistic):           0.000366
Time:                        20:00:46   Log-Likelihood:                -37.758
No. Observations:                  69   AIC:                             79.52
Df Residuals:                      67   BIC:                             83.99
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.4028      0.107     

In [12]:
print('Intercept       & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(intercept_logit[20], intercept_linear[20], intercept_total[20], intercept_logit[25], intercept_linear[25], intercept_total[25], intercept_logit[30], intercept_linear[30], intercept_total[30]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_intercept_logit[20], t_intercept_total[20], t_intercept_logit[25], t_intercept_total[25], t_intercept_logit[30], t_intercept_total[30]))
print('Bearish (dummy) & & {}   & {} & & {}   & & {}   & {} & & {}   & & {}   & {} & & {}'.format(beta_logit[20], beta_linear[20], beta_total[20], beta_logit[25], beta_linear[25], beta_total[25], beta_logit[30], beta_linear[30], beta_total[30]))
print('                & & ({}) & -  & & ({}) & & ({}) & -  & & ({}) & & ({}) & -  & & ({})'.format(t_beta_logit[20], t_beta_total[20], t_beta_logit[25], t_beta_total[25], t_beta_logit[30], t_beta_total[30]))
print('Adj. $R^2$ & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {} & & \multicolumn2c{} & & {}'.format(r2[20], r2_total[20], r2[25], r2_total[25], r2[30], r2_total[30]))



Intercept       & & -3.135   & 0.042 & & 475.113   & & -1.335   & 0.208 & & 488.819   & & -0.511   & 0.375 & & 491.681
                & & (-3.069) & -  & & (38.788) & & (-2.656) & -  & & (54.924) & & (-1.212) & -  & & (49.298)
Bearish (dummy) & & 2.912   & 0.403 & & -156.201   & & 2.182   & 0.492 & & -209.171   & & 3.15   & 0.558 & & -216.648
                & & (2.736) & -  & & (-10.298) & & (3.403) & -  & & (-17.518) & & (2.818) & -  & & (-13.471)
Adj. $R^2$ & & \multicolumn2c0.173 & & 0.607 & & \multicolumn2c0.181 & & 0.852 & & \multicolumn2c0.259 & & 0.826
